# Function Calling
* Data can be made available to LLM by following 3 methods.
    * Providing required data to base model of LLM (by fine tuning)
    * By saving the data on a vector data base (in the form of embeddings(numbers))
    * Function calling
        * Used to get real time, sensitive data (data remains at the user end)
        * LLM calls the API provided by the user to provide the answer to the user
        * As OperAI is stateless, it has to call the function every time the user asks the same question

In [ ]:
%pip install -r requirements.txt

In [2]:
from openai import OpenAI
import json
from dotenv import find_dotenv, load_dotenv

_ : bool = load_dotenv(find_dotenv())

client : OpenAI = OpenAI()

In [14]:
def get_current_weather(location: str, unit:str='fahrenheit') -> str:
    """Get current weather in a given location"""
    if "tokio" in location.lower():
        return json.dumps({"Location": "Tokio", "temperature": "10", "unit": "celcius"})
    elif "san fransisco" in location.lower():
        return json.dumps({"location": "San Fransisco", "temperature": "15", "unit": "fahrenheit"})
    elif "paris" in location.lower():
        return json.dumps({"location": "paris", "temperature": "18", "unit": "celcuis"})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})


In [17]:
from openai.types.chat.chat_completion import ChatCompletionMessage, ChatCompletion

def run_conversation(main_request: str)->str:
    # step-1: send conversation and available functions to the model
    messages = [{"role": "user", "content": main_request}]

    # functions details
    tools = [
        {
            "type":"function",
            "function": {
                "name": "get_current_weather",
                # description is important because NLP understands the context of question by reading it
                "description": "Get the current weather in a given location",
                # parametres required by "get_current_weather" function
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and the state, e.g. San Fransisco, CA",
                        },
                        "Unit": {"type":"string", "enum":["celsius", "fahrenheit"]},
                    },
                    "required": ["location"],
                }
            }
        }
    ]

    # first request
    response: ChatCompletion = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",  
        messages=messages,
        tools=tools,        # for function calling we require tools
        tool_choice="auto"  # auto is default, but we will be explicit
    )

    response_message: ChatCompletionMessage = response.choices[0].message
    display("* First Response: ", dict(response_message))

    tool_calls = response_message.tool_calls    # will contain all the functions that are to be called
    display("*First Response Tool Calls: ", list(tool_calls))

    # step-2: check if the model wanted to call a function

    if tool_calls:
        # step-3: call the function
        # Note: the JSON response may not always be valid; be sure to handle the erros
        available_functions = {
            "get_current_weather":get_current_weather   # only one function in this example, can be multiple
        }

    messages.append(response_message)   # extends conversation with assistsnt's resply

    # srep-4: send the info for each function call and function response to the model

    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_to_call = available_functions[function_name]
        function_args = json.loads(tool_call.function.arguments)
        function_response = function_to_call(
            location = function_args.get("location"),
            unit=function_args.get("unit")
        )
        messages.append(
            {
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": function_response
            }
        )   # extend conversation with function response

    display("* Second Resonse Messages: ", list(messages))

    second_response: ChatCompletion = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=messages,
    )   # Get a new response from the model where it can see the function response

    print("* Second Response: ", dict(second_response))

    return second_response.choices[0].message.content

In [ ]:
run_conversation("What is current weather in tokio?")